## Env

In [1]:
# .env 환경설정 파일을 읽어서 환경변수로 설정
from dotenv import load_dotenv
load_dotenv()

True

## HWP 문서 가져오기

https://nomadamas.gitbook.io/ragchain-docs/ragchain-structure/file-loader/win32-hwp-loader

In [2]:
from RAGchain.preprocess.loader import Win32HwpLoader

loader = Win32HwpLoader("(첨부5) 2024년도 하반기 중남미 지역기구 파견인턴 선발 공고.hwpx")
documents = loader.load()

len(documents)

c:\Users\redwi\AppData\Local\pypoetry\Cache\virtualenvs\sh-vietnam-_0ZvXL4d-py3.11\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


10

In [3]:
from pprint import pprint
pprint(documents[0].page_content)

('2024년도 하반기 중남미 지역기구 파견 인턴 선발 공고1. 선발 개요  가. 공통 자격요건   ㅇ 대한민국 국적자로, 국내외 '
 '대학생(3-4학년) 또는 대학원생   ㅇ 영어 또는 스페인어로 업무처리가 가능한 자   나. 기구별 선발 기간 및 인원   ㅇ 파견기간은 '
 '원칙적으로 4~6개월이나, 기구 사정에 따라 변동 가능       - OAS(미주기구)의 경우 선발 후 25년 상반기 파견   ㅇ 기구별 '
 '선호 상세 지원자격은 본 공고 4페이지 “참고1” 참조    다. 지원 내역 : 왕복 항공료(정액), 현지 정착비 및 월 체재비     '
 '※ 다만, 지원금으로 충당할 수 없는 파견 기구 및 파견국의 물가 등에 따라 발생하는 기타 비용은 지원자 부담 필요 2. 선발 일정   '
 'ㅇ 2024.5.30.(목) 18:00서류접수 마감(이메일 접수)    ㅇ 2024.5.31.(금)  서류전형 합격자 발표(개별 '
 '통보)   ㅇ 2024.6.4.(화)       필기시험 및 면접시험(상세내용 추후 공지)   ㅇ 2024.6.10.(월) 외교부 추천 '
 '후보자 발표(개별 통보)   ㅇ 2024.6.11.(화) ~기구측에 우리측 후보자 추천   ㅇ 2024.6월~2024.7월  최종 합격자 '
 '발표(개별 통보)     3. 서류 접수 안내   ㅇ 외교부 중남미협력과 이메일(latinca@mofa.go.kr)로 관련 서류 '
 '송부    - 국문 신청서 : 한글 파일(*hwp) 또는 PDF 파일         ※ 지원자는 국문 신청서에 지원기구 1순위 및 2순위 '
 '기재 요망         - ▵영문 신청서(Application for Internship, Curriculum Vitae) 각 1부, '
 '▵영문 재학 또는 휴학증명서, ▵영문 대학 및 대학원 성적증명서, ▵기타 증명서 : 1개의 컬러 PDF 파일로 변환하여 송부     - '
 '영문 교수 추천서 1부 : 추천 교수가 직접 외교부 중남미협력과 이메일로 송부    

In [4]:
pprint(documents[1].page_content)

('<table\n'
 '                    border="1"\n'
 '                    width="50%"\n'
 '                    height="200"\n'
 '                    cellspacing="5">\n'
 '\t <tr>\n'
 '\t \t <td>(학력)※ 위 학력사항은 기본 요건이며, 기구별 상세요건 확인 요망※ 기구별 학력요건은 본 공고 4페이지 “참고1” '
 '참조(어학)※ 어학능력 증빙 자료(어학성적, 자격증 등) 제출 요망 ※ 증빙자료가 없을 시 언어 능력 뒷받침할 수 있는 내용을 신청서에 '
 '기재※ 기구별 어학요건은 본 공고 4페이지 “참고1” 참조(기타 고려사항)※ 대한민국 정부가 지원하는 유사 인턴 프로그램 기참가자는 지원 '
 '불가※ 비수도권대학 학생의 지원 장려(총 인원의 30%이상 비수도권대학에서 선발할 계획. 다만, 비수도권대학 학생 지원율 저조 등 사유가 '
 '있는 경우 동 목표 비율은 탄력적으로 조정될 예정)   *비수도권대학 판단 기준은 국가법령정보센터(www.law.go.kr)에 게시된 '
 '‘균형인사지침’ 참고※ 백신 접종 완료 여부 확인을 위한 백신접종완료증명서 사본 제출</td>\n'
 '\t </tr>\n'
 '</table>')


## Simple RAG

#### Indexing

In [5]:
from langchain_community.vectorstores import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.chat_models import ChatOllama
from langchain.prompts import ChatPromptTemplate

In [6]:
# HugoingFace Embeddings를 다운로드
from langchain.embeddings import HuggingFaceEmbeddings

embedding_model_name = "BM-K/KoSimCSE-roberta-multitask"

embeddings_model = HuggingFaceEmbeddings(
    model_name=embedding_model_name,  
)

# HugoingFace Embedding 모델의 Tokenizer를 사용하여 토큰화
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained(embedding_model_name)

c:\Users\redwi\AppData\Local\pypoetry\Cache\virtualenvs\sh-vietnam-_0ZvXL4d-py3.11\Lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(
No sentence-transformers model found with name BM-K/KoSimCSE-roberta-multitask. Creating a new one with mean pooling.


In [7]:
embeddings_model

HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 512, 'do_lower_case': False}) with Transformer model: RobertaModel 
  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
), model_name='BM-K/KoSimCSE-roberta-multitask', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False, show_progress=False)

In [8]:
# Token 수를 기준으ㄹ 문서를 청크 단위로 분할
text_splitter = RecursiveCharacterTextSplitter.from_huggingface_tokenizer(
    tokenizer = tokenizer,
    chunk_size = 500,
    chunk_overlap  = 50,
)

split_docs = text_splitter.split_documents(documents)
print(len(split_docs))
print(split_docs[0])

17
page_content='2024년도 하반기 중남미 지역기구 파견 인턴 선발 공고1. 선발 개요  가. 공통 자격요건   ㅇ 대한민국 국적자로, 국내외 대학생(3-4학년) 또는 대학원생   ㅇ 영어 또는 스페인어로 업무처리가 가능한 자   나. 기구별 선발 기간 및 인원   ㅇ 파견기간은 원칙적으로 4~6개월이나, 기구 사정에 따라 변동 가능       - OAS(미주기구)의 경우 선발 후 25년 상반기 파견   ㅇ 기구별 선호 상세 지원자격은 본 공고 4페이지 “참고1” 참조    다. 지원 내역 : 왕복 항공료(정액), 현지 정착비 및 월 체재비' metadata={'source': '(첨부5) 2024년도 하반기 중남미 지역기구 파견인턴 선발 공고.hwpx', 'page_type': 'text'}


In [9]:
vectorstore = Chroma.from_documents(documents=split_docs, 
                                    embedding=embeddings_model)

#### Retreival

In [10]:
# VectorStore를 사용하여 검색기 생성
retriever = vectorstore.as_retriever(search_kwargs={'k': 2})

# Prompt
template = """Answer the question based only on the following context:
{context}

Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

# Chat Model
llm = ChatOllama(model="gemma2")


def format_docs(docs):
    # 중복을 제거
    unique_contents = set(doc.page_content for doc in docs)
    
    # \n\n으로 연결하여 문자열로 반환
    return "\n\n".join(unique_contents)


# RAG Chain
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

# Chain 실행
response = rag_chain.invoke("파견 기간은 얼마인가요?")
print(response)

파견 기간은 원칙적으로 4~6개월이나, 기구 사정에 따라 변동 가능합니다. 





In [11]:
retriever.get_relevant_documents("파견 기간은 얼마인가요?")

c:\Users\redwi\AppData\Local\pypoetry\Cache\virtualenvs\sh-vietnam-_0ZvXL4d-py3.11\Lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


[Document(metadata={'page_type': 'text', 'source': '(첨부5) 2024년도 하반기 중남미 지역기구 파견인턴 선발 공고.hwpx'}, page_content='2024년도 하반기 중남미 지역기구 파견 인턴 선발 공고1. 선발 개요  가. 공통 자격요건   ㅇ 대한민국 국적자로, 국내외 대학생(3-4학년) 또는 대학원생   ㅇ 영어 또는 스페인어로 업무처리가 가능한 자   나. 기구별 선발 기간 및 인원   ㅇ 파견기간은 원칙적으로 4~6개월이나, 기구 사정에 따라 변동 가능       - OAS(미주기구)의 경우 선발 후 25년 상반기 파견   ㅇ 기구별 선호 상세 지원자격은 본 공고 4페이지 “참고1” 참조    다. 지원 내역 : 왕복 항공료(정액), 현지 정착비 및 월 체재비'),
 Document(metadata={'page_type': 'table', 'source': '(첨부5) 2024년도 하반기 중남미 지역기구 파견인턴 선발 공고.hwpx'}, page_content='<table\n                    border="1"\n                    width="50%"\n                    height="200"\n                    cellspacing="5">\n\t <tr>\n\t \t <td>기구명</td>\n\t \t <td>파견지</td>\n\t \t <td>파견 기간</td>\n\t \t <td>인원</td>\n\t </tr>\n\t <tr>\n\t \t <td>PARLATINO(라틴아메리카의회)</td>\n\t \t <td>파나마</td>\n\t \t <td>6개월</td>\n\t \t <td>1</td>\n\t </tr>\n\t <tr>\n\t \t <td>IOM(국제이주기구)</td>\n\t \t <td>페루</td>\n\t \t <td>6개월</td>\n\t \t <td>1</td>\n\t </tr>\n\t <tr>\n\t \t 

In [12]:
response = rag_chain.invoke("필기시험이 있는 날짜는 언제인가요?")
print(response)

2024.6.4.(화)  



In [13]:
retriever.get_relevant_documents("필기시험이 있는 날짜는 언제인가요?")

[Document(metadata={'page_type': 'text', 'source': '(첨부5) 2024년도 하반기 중남미 지역기구 파견인턴 선발 공고.hwpx'}, page_content='왕복 항공료(정액), 현지 정착비 및 월 체재비     ※ 다만, 지원금으로 충당할 수 없는 파견 기구 및 파견국의 물가 등에 따라 발생하는 기타 비용은 지원자 부담 필요 2. 선발 일정   ㅇ 2024.5.30.(목) 18:00서류접수 마감(이메일 접수)    ㅇ 2024.5.31.(금)  서류전형 합격자 발표(개별 통보)   ㅇ 2024.6.4.(화)       필기시험 및 면접시험(상세내용 추후 공지)   ㅇ 2024.6.10.(월) 외교부 추천 후보자 발표(개별 통보)   ㅇ 2024.6.11.(화) ~기구측에 우리측 후보자 추천   ㅇ'),
 Document(metadata={'page_type': 'table', 'source': '(첨부5) 2024년도 하반기 중남미 지역기구 파견인턴 선발 공고.hwpx'}, page_content='<table\n                    border="1"\n                    width="50%"\n                    height="200"\n                    cellspacing="5">\n\t <tr>\n\t \t <td>※ 외교부 추천 후보자에 대해 각 지역기구가 최종 합격 여부를 결정하며, 동 과정에서 후보자가 합격하지 못할 수 있음※ 상기 일정은 외교부 및 인턴 접수 기구의 사정에 의해 변경될 수 있음※ 기구측에 추천되는 후보자들은 이메일 접수서류의 원본을 제출해야 하며, 제출된 서류는 일체 반환하지 않음※ 필기시험 평가 항목은 5페이지 “참고2” 참조</td>\n\t </tr>\n</table>')]

In [14]:
response = rag_chain.invoke("국제이주기구 파견은 언제 하나요?")
print(response)

This document does not contain information about the International Organization for Migration (IOM). 





In [15]:
retriever.get_relevant_documents("국제이주기구 파견은 언제 하나요?")

[Document(metadata={'page_type': 'table', 'source': '(첨부5) 2024년도 하반기 중남미 지역기구 파견인턴 선발 공고.hwpx'}, page_content='<table\n                    border="1"\n                    width="50%"\n                    height="200"\n                    cellspacing="5">\n\t <tr>\n\t \t <td>※ 상기 지역기구 외에도 기타 지역기구로부터 인턴 수요가 추가적으로 발생 가능※ 각 지역기구별 적격자가 없다고 판단되는 경우 선발하지 않을 수 있음※ 치안 등 지역기구 소재지 내 제반 사정에 따라 일정 조정 등 변동 가능 ※ 각 지역 기구에 관한 상세 내용은 ‘외교부 홈페이지(www.mofa.go.kr)>외교정책>국제기구‧지역협력체>중남미지역기구’ 참고</td>\n\t </tr>\n</table>'),
 Document(metadata={'page_type': 'text', 'source': '(첨부5) 2024년도 하반기 중남미 지역기구 파견인턴 선발 공고.hwpx'}, page_content='2024년도 하반기 중남미 지역기구 파견 인턴 선발 공고1. 선발 개요  가. 공통 자격요건   ㅇ 대한민국 국적자로, 국내외 대학생(3-4학년) 또는 대학원생   ㅇ 영어 또는 스페인어로 업무처리가 가능한 자   나. 기구별 선발 기간 및 인원   ㅇ 파견기간은 원칙적으로 4~6개월이나, 기구 사정에 따라 변동 가능       - OAS(미주기구)의 경우 선발 후 25년 상반기 파견   ㅇ 기구별 선호 상세 지원자격은 본 공고 4페이지 “참고1” 참조    다. 지원 내역 : 왕복 항공료(정액), 현지 정착비 및 월 체재비')]

In [16]:
retriever.get_relevant_documents("IOM")

[Document(metadata={'page_type': 'table', 'source': '(첨부5) 2024년도 하반기 중남미 지역기구 파견인턴 선발 공고.hwpx'}, page_content='<table\n                    border="1"\n                    width="50%"\n                    height="200"\n                    cellspacing="5">\n\t <tr>\n\t \t <td>기구명</td>\n\t \t <td>파견지</td>\n\t \t <td>파견 기간</td>\n\t \t <td>인원</td>\n\t </tr>\n\t <tr>\n\t \t <td>PARLATINO(라틴아메리카의회)</td>\n\t \t <td>파나마</td>\n\t \t <td>6개월</td>\n\t \t <td>1</td>\n\t </tr>\n\t <tr>\n\t \t <td>IOM(국제이주기구)</td>\n\t \t <td>페루</td>\n\t \t <td>6개월</td>\n\t \t <td>1</td>\n\t </tr>\n\t <tr>\n\t \t <td>OPANAL(중남미핵무기금지기구)</td>\n\t \t <td>멕시코</td>\n\t \t <td>2024.7월 ~ 11월</td>\n\t \t <td>1</td>\n\t </tr>\n\t <tr>\n\t \t <td>GGGI(글로벌녹색성장연구소)</td>\n\t \t <td>파라과이</td>\n\t \t <td>추후 조율</td>\n\t \t <td>1</td>\n\t </tr>\n\t <tr>\n\t \t <td>SEGIB(이베로아메리카사무국)</td>'),
 Document(metadata={'page_type': 'table', 'source': '(첨부5) 2024년도 하반기 중남미 지역기구 파견인턴 선발 공고.hwpx'}, page_content='<table\n                    border

## Table Data

In [17]:
for d in split_docs:
    if "IOM" in d.page_content:
        print(d.page_content)
        sample_text = d.page_content
        break

<table
                    border="1"
                    width="50%"
                    height="200"
                    cellspacing="5">
	 <tr>
	 	 <td>기구명</td>
	 	 <td>파견지</td>
	 	 <td>파견 기간</td>
	 	 <td>인원</td>
	 </tr>
	 <tr>
	 	 <td>PARLATINO(라틴아메리카의회)</td>
	 	 <td>파나마</td>
	 	 <td>6개월</td>
	 	 <td>1</td>
	 </tr>
	 <tr>
	 	 <td>IOM(국제이주기구)</td>
	 	 <td>페루</td>
	 	 <td>6개월</td>
	 	 <td>1</td>
	 </tr>
	 <tr>
	 	 <td>OPANAL(중남미핵무기금지기구)</td>
	 	 <td>멕시코</td>
	 	 <td>2024.7월 ~ 11월</td>
	 	 <td>1</td>
	 </tr>
	 <tr>
	 	 <td>GGGI(글로벌녹색성장연구소)</td>
	 	 <td>파라과이</td>
	 	 <td>추후 조율</td>
	 	 <td>1</td>
	 </tr>
	 <tr>
	 	 <td>SEGIB(이베로아메리카사무국)</td>


#### HTML 테이블 구조를 컨텍스트로 전달

In [18]:
sample_text

'<table\n                    border="1"\n                    width="50%"\n                    height="200"\n                    cellspacing="5">\n\t <tr>\n\t \t <td>기구명</td>\n\t \t <td>파견지</td>\n\t \t <td>파견 기간</td>\n\t \t <td>인원</td>\n\t </tr>\n\t <tr>\n\t \t <td>PARLATINO(라틴아메리카의회)</td>\n\t \t <td>파나마</td>\n\t \t <td>6개월</td>\n\t \t <td>1</td>\n\t </tr>\n\t <tr>\n\t \t <td>IOM(국제이주기구)</td>\n\t \t <td>페루</td>\n\t \t <td>6개월</td>\n\t \t <td>1</td>\n\t </tr>\n\t <tr>\n\t \t <td>OPANAL(중남미핵무기금지기구)</td>\n\t \t <td>멕시코</td>\n\t \t <td>2024.7월 ~ 11월</td>\n\t \t <td>1</td>\n\t </tr>\n\t <tr>\n\t \t <td>GGGI(글로벌녹색성장연구소)</td>\n\t \t <td>파라과이</td>\n\t \t <td>추후 조율</td>\n\t \t <td>1</td>\n\t </tr>\n\t <tr>\n\t \t <td>SEGIB(이베로아메리카사무국)</td>'

In [19]:
len(tokenizer.encode(sample_text))

351

In [20]:
llm.invoke(f"""Answer the question based only on the following context:
{sample_text}

Question: 국제이주기구 파견은 언제 하나요?
""")

AIMessage(content='6개월  \n', response_metadata={'model': 'gemma2', 'created_at': '2024-07-03T06:51:32.0463146Z', 'message': {'role': 'assistant', 'content': ''}, 'done_reason': 'stop', 'done': True, 'total_duration': 1042553600, 'load_duration': 2238200, 'prompt_eval_count': 386, 'prompt_eval_duration': 690969000, 'eval_count': 6, 'eval_duration': 346193000}, id='run-7b3b9a1a-d843-4522-80c0-bf8f9d4b09a8-0')

#### Markdown 형식으로 테이블 데이터를 변환하여 컨텍스트로 전달

In [21]:
import pandas as pd
from bs4 import BeautifulSoup

# BeautifulSoup을 사용하여 HTML 파싱
soup = BeautifulSoup(sample_text, 'html.parser')

# 테이블 찾기
table = soup.find('table')

# Pandas DataFrame으로 변환
df = pd.read_html(str(table))[0]

# DataFrame을 Markdown으로 변환
markdown_table = df.to_markdown(index=False)

print(markdown_table)

| 0                            | 1        | 2               | 3    |
|:-----------------------------|:---------|:----------------|:-----|
| 기구명                       | 파견지   | 파견 기간       | 인원 |
| PARLATINO(라틴아메리카의회)  | 파나마   | 6개월           | 1    |
| IOM(국제이주기구)            | 페루     | 6개월           | 1    |
| OPANAL(중남미핵무기금지기구) | 멕시코   | 2024.7월 ~ 11월 | 1    |
| GGGI(글로벌녹색성장연구소)   | 파라과이 | 추후 조율       | 1    |
| SEGIB(이베로아메리카사무국)  | nan      | nan             | nan  |


C:\Users\redwi\AppData\Local\Temp\ipykernel_15016\272752229.py:11: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(table))[0]


In [22]:
llm.invoke(f"""Answer the question based only on the following context:
{markdown_table}

Question: 국제이주기구 파견은 언제 하나요?
""")

AIMessage(content='6개월 동안 파견됩니다.  \n', response_metadata={'model': 'gemma2', 'created_at': '2024-07-03T06:51:40.3430366Z', 'message': {'role': 'assistant', 'content': ''}, 'done_reason': 'stop', 'done': True, 'total_duration': 1257063500, 'load_duration': 2128700, 'prompt_eval_count': 233, 'prompt_eval_duration': 562938000, 'eval_count': 11, 'eval_duration': 689807000}, id='run-e8c23ab4-4835-481d-b14f-d9e558b6b371-0')

In [23]:
len(tokenizer.encode(markdown_table))

203

## Text 와 Table 구분하여 청크로 분할

In [27]:
import pandas as pd
from bs4 import BeautifulSoup

def convert_html_table_to_markdown(html_table_text: str) -> str:

    # BeautifulSoup을 사용하여 HTML 파싱
    soup = BeautifulSoup(html_table_text, 'html.parser')

    # 테이블 찾기
    table = soup.find('table')

    # Pandas DataFrame으로 변환
    df = pd.read_html(str(table))[0]

    # DataFrame을 Markdown으로 변환
    markdown_table = df.to_markdown(index=False)

    return markdown_table

In [24]:
documents[0].metadata

{'source': '(첨부5) 2024년도 하반기 중남미 지역기구 파견인턴 선발 공고.hwpx', 'page_type': 'text'}

In [30]:
markdown_documents = []

for doc in documents:
    if doc.metadata.get("page_type")=='table':

        try:
            doc.page_content = convert_html_table_to_markdown(doc.page_content)
            markdown_documents.append(doc)
        except:
            markdown_documents.append(doc)
    else:
        markdown_documents.append(doc)


len(markdown_documents)
            

C:\Users\redwi\AppData\Local\Temp\ipykernel_15016\2485168362.py:13: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(table))[0]


10

In [31]:
new_split_docs = text_splitter.split_documents(markdown_documents)
print(len(new_split_docs))
print(new_split_docs[0])

Token indices sequence length is longer than the specified maximum sequence length for this model (694 > 512). Running this sequence through the model will result in indexing errors


26
page_content='2024년도 하반기 중남미 지역기구 파견 인턴 선발 공고1. 선발 개요  가. 공통 자격요건   ㅇ 대한민국 국적자로, 국내외 대학생(3-4학년) 또는 대학원생   ㅇ 영어 또는 스페인어로 업무처리가 가능한 자   나. 기구별 선발 기간 및 인원   ㅇ 파견기간은 원칙적으로 4~6개월이나, 기구 사정에 따라 변동 가능       - OAS(미주기구)의 경우 선발 후 25년 상반기 파견   ㅇ 기구별 선호 상세 지원자격은 본 공고 4페이지 “참고1” 참조    다. 지원 내역 : 왕복 항공료(정액), 현지 정착비 및 월 체재비' metadata={'source': '(첨부5) 2024년도 하반기 중남미 지역기구 파견인턴 선발 공고.hwpx', 'page_type': 'text'}


In [32]:
vectorstore2 = Chroma.from_documents(documents=new_split_docs, 
                                    embedding=embeddings_model)


retriever2 = vectorstore2.as_retriever(search_kwargs={'k': 2})

rag_chain2 = ( 
    {"context": retriever2 | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

response = rag_chain2.invoke("국제이주기구 파견은 언제 하나요?")
response

'IOM(국제이주기구) 파견은 6개월 동안 페루에서 이루어집니다.  \n'

In [33]:
retriever2.get_relevant_documents("국제이주기구 파견은 언제 하나요?")

[Document(metadata={'page_type': 'table', 'source': '(첨부5) 2024년도 하반기 중남미 지역기구 파견인턴 선발 공고.hwpx'}, page_content='| 0                            | 1                | 2                   | 3    |\n|:-----------------------------|:-----------------|:--------------------|:-----|\n| 기구명                       | 파견지           | 파견 기간           | 인원 |\n| PARLATINO(라틴아메리카의회)  | 파나마           | 6개월               | 1    |\n| IOM(국제이주기구)            | 페루             | 6개월               | 1    |\n| OPANAL(중남미핵무기금지기구) | 멕시코           | 2024.7월 ~ 11월     | 1    |\n| GGGI(글로벌녹색성장연구소)   | 파라과이         | 추후 조율           | 1    |\n| SEGIB(이베로아메리카사무국)  | 스페인           | 2024.9월 ~ 2025.2월 | 1    |\n| CAF(중남미개발은행)          | 콜롬비아         | 2024.9월 ~ 2025.3월 | 1    |\n| ACTO(아마존협력조약기구)     | 브라질           | 6개월               | 1    |\n| OAS(미주기구)                | 미국(워싱턴)     | 2025.1월 ~ 2025.7월 | 1    |\n| ACS(카리브국가연합)          | 트리니다드토바고 | 6개월               | 1    |'),
 Document(metadata={'page_type': 'table'